In [ ]:
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path, skiprows):
    return np.loadtxt(path, dtype=float, skiprows=skiprows)

def readFileFromDialog(skiprows):
    return readFile(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Average displacement

In [ ]:
traveledData = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(len(traveledData[0]),2,squeeze=False, figsize = (10,6*len(traveledData[0])), dpi = 100)
Means = np.zeros(len(traveledData[0]))
for i in range(len(traveledData[0])):
    Ax[i,0].plot(traveledData[1:,i],'.',label='y={}'.format(traveledData[0,i]))
    Means[i] = np.mean(traveledData[1:,i])
    Ax[i,0].plot(Means[i]*np.ones(len(traveledData[1:,i])))
    Ax[i,0].legend()
    n,bins,patches = Ax[i,1].hist(traveledData[1:,i],bins = 41, density = True)
    Ax[i,1].vlines(Means[i],0,np.max(n),color='darkorange')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,5), dpi = 100)
Ax[0,0].plot(Means,traveledData[0],'.')
Ax[0,0].set_xlabel('average $\Delta x / \sigma$ per sweep')
Ax[0,0].set_ylabel('y / $\sigma$')
#Ax[0,0].set_ylim(0,1)
popt, pcov = curve_fit(LinearFunction, Means[:], traveledData[0][:])
print(popt)
print(popt[1]/popt[0])
print('shear rate: {:.3E}'.format(1/popt[0]))
Ax[0,0].plot(Means[:],LinearFunction(Means[:],*popt))
Ax[0,0].set_title('Average $\Delta x$ vs y')
Ax[0,0].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
#Fig.savefig("plots/shear_analysis/avg_delta_x_field_no_lees_edwards_T=1.png",format='png')

## Avg MSD

In [ ]:
AvgMSDSeries = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
xdata = np.arange(1,len(AvgMSDSeries)+1)
Ax[0,0].plot(xdata,AvgMSDSeries/xdata,'.')
popt, pcov = curve_fit(lambda x,n: LinearFunction(x,0.0,n), xdata[:9], AvgMSDSeries[:9]/xdata[:9])
print(popt)
Ax[0,0].plot(xdata[:9],LinearFunction(xdata[:9],0.0,*popt))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')

## Stress plotting

In [ ]:
NumberOfSubdivisions = 14

In [ ]:
xStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3],'_',label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3]))
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4],'_',label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg = {:.4f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in x direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
Mean = 0
for i in range(NumberOfSubdivisions-1):
    Mean += np.mean(xStressData[NumberOfSubdivisions-i-2::NumberOfSubdivisions,4])
print("Avg shear stress except top layer: {:.3f}".format(Mean/(NumberOfSubdivisions-1)))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='y={}, Avg = {:.4f}'.format(xStressData[NumberOfSubdivisions-1-i,0],MeanShear))
    Ax[0,0].legend()

In [ ]:
yStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,yStressData[i::NumberOfSubdivisions,3],'_',label='x={}'.format(yStressData[i,0]))
    MeanNormal = np.mean(yStressData[i::NumberOfSubdivisions,3])
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*MeanNormal)
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,yStressData[i::NumberOfSubdivisions,4],'_',label='x={}'.format(yStressData[i,0]))
    MeanShear = np.mean(yStressData[i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg={:.3f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in y direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
print("Avg pressure: {:.4f}".format((np.mean(xStressData[:,3])+np.mean(yStressData[:,3]))*0.5))
print("Avg shear stress x direction: {:.4f}".format(np.mean(xStressData[:,4])))
print("Avg shear stress y direction: {:.4f}".format(np.mean(yStressData[:,4])))

## Series plotting

In [ ]:
NumberOfASeries = readFileFromDialog()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(NumberOfASeries,'.')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].hist(NumberOfASeries,bins = 41, density = True)
Ax[0,0].set_xlabel(r'$N_A$')
Ax[0,0].set_ylabel('number of occurences')

In [ ]:
PotEnergySeries = readFileFromDialog()

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(PotEnergySeries,'.')

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(5,5))
Ax[0,0].hist(PotEnergySeries,bins = 41, density = True)
Ax[0,0].set_xlabel(r'$N_A$')
Ax[0,0].set_ylabel('number of occurences')

## State plotting

In [ ]:
APositions, BPositions = readParticleConfigFromDialog(0)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b',ms=4)
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)
#Fig.savefig('ParticleConfig_T=0.52_N=16000_multiple_phases present'+'.png',format='png')